In [9]:
!pip install tweet-preprocessor

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #regular expressions
import string 

from nltk.corpus import stopwords
from nltk import word_tokenize
import preprocessor as p
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

reuse = False

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-19 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-16 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-23 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-20 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-27 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-25 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-24 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-26 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-29 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-22 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/2020-04-17 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-late-april/20

In [11]:
def join_df(path):
    data = pd.DataFrame()
    for sub_path in os.listdir(path):
        print(f"In path : {path}{sub_path}")
        for file in os.listdir(path+sub_path):
            temp = pd.read_csv(path+sub_path+'/'+file)
            temp = temp[(temp.country_code=='IN') & (temp.lang=='en')]
            data = pd.concat([data,temp])
    return data
        

In [12]:
path = '/kaggle/input/'

if os.path.exists('/kaggle/working/main.csv'):    
    print('Loading existing dataset')
    df = pd.read_csv('/kaggle/working/main.csv')
    reuse = True
else:
    print('Creating main dataset')
    df = join_df(path)
    df.to_csv("/kaggle/working/main.csv")
    
    
print(df.head())
print(df.shape)


Loading existing dataset
   Unnamed: 0            status_id             user_id            created_at  \
0         679  1251662584148717568            89277767  2020-04-19T00:03:09Z   
1        1915  1251664388819312640          1133202589  2020-04-19T00:10:19Z   
2        2096  1251664626196111361           105498377  2020-04-19T00:11:16Z   
3        2778  1251665538838925312  954747551244107777  2020-04-19T00:14:53Z   
4        3562  1251666576488697857           140564770  2020-04-19T00:19:01Z   

       screen_name                                               text  \
0  shamanthcricket  I so badly want #India and an #Indian to find ...   
1    Elaine_Dsouza  #PhysicalDistancing means finding new ways to ...   
2   swadhinasomiya  I would urge CM @Neiphiu_Rio to look into the ...   
3         iamzam08  @ladygaga 😍😍😍 \nBut apne ko tow WORLD BIGGEST ...   
4        Cool_jack  Switzerland projected flags of various countri...   

                source  reply_to_status_id  reply_to_us

In [13]:
# checking for missing values
df.isnull().sum()

Unnamed: 0                  0
status_id                   0
user_id                     0
created_at                  0
screen_name                 0
text                        0
source                      0
reply_to_status_id      36725
reply_to_user_id        34670
reply_to_screen_name    34670
is_quote                    0
is_retweet                  0
favourites_count            0
retweet_count               0
country_code                0
place_full_name             0
place_type                  0
followers_count             0
friends_count               0
account_lang            41618
account_created_at          0
verified                    0
lang                        0
dtype: int64

In [14]:
#df.head()
for i in df.columns:
    if i!='text':
        df.drop(i,axis=1,inplace=True)
print(df.head())
df.dropna(axis=0,inplace=False)

                                                text
0  I so badly want #India and an #Indian to find ...
1  #PhysicalDistancing means finding new ways to ...
2  I would urge CM @Neiphiu_Rio to look into the ...
3  @ladygaga 😍😍😍 \nBut apne ko tow WORLD BIGGEST ...
4  Switzerland projected flags of various countri...


,text
0,I so badly want #India and an #Indian to find ...
1,#PhysicalDistancing means finding new ways to ...
2,I would urge CM @Neiphiu_Rio to look into the ...
3,@ladygaga 😍😍😍 \nBut apne ko tow WORLD BIGGEST ...
4,Switzerland projected flags of various countri...
...,...
41613,Time for us @xfapsiansdelhi to get together &a...
41614,@raghav_chadha @ArvindKejriwal Now people will...
41615,"THE VIRUS DOESN'T MOVE,\nPEOPLE MOVE IT.\n\nWE..."
41616,"@JohnCena Stay home, stay safe @JohnCena becau..."


In [15]:
#cleaning the tweets function

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)



In [16]:
#replace tabs and spaces

df['text']= df['text'].replace({'\n':" ","\t":" "})

for i in range(len(df['text'])):
    df['text'][i]=clean_tweets(df['text'][i])
    #print(df['text'])

df['text']

0        I badly want India Indian find vaccine coronav...
1        PhysicalDistancing means finding new ways conn...
2        I would urge CM Neiphiu_Rio look matter direct...
3        ladygaga 😍😍😍 But apne ko tow WORLD BIGGEST SUP...
4        Switzerland projected flags various countries ...
                               ...                        
41613    Time us xfapsiansdelhi get together amp collec...
41614    raghav_chadha ArvindKejriwal Now people die hu...
41615    THE VIRUS DOES N'T MOVE PEOPLE MOVE IT WE STOP...
41616    JohnCena Stay home stay safe JohnCena U ca n't...
41617    Time us xfapsiansdelhi get together amp collec...
Name: text, Length: 41618, dtype: object

In [17]:
#getting sentiment scores and polarity
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    return [blob.sentiment.polarity, blob.sentiment.subjectivity]




In [18]:
polarity,subjectivity = [], []
for i in range(len(df)):
    pol , sub = get_sentiment(df['text'][i])
    polarity.append(pol)
    subjectivity.append(sub)
    
df['polarity'] = polarity
df['subjectivity'] = subjectivity
df.head()

,text,polarity,subjectivity
0,I badly want India Indian find vaccine coronav...,-0.700000,0.666667
1,PhysicalDistancing means finding new ways conn...,0.278788,0.751515
2,I would urge CM Neiphiu_Rio look matter direct...,0.050000,0.700000
3,ladygaga 😍😍😍 But apne ko tow WORLD BIGGEST SUP...,0.000000,0.000000
4,Switzerland projected flags various countries ...,0.100000,0.333333


In [19]:
l=len(df['polarity'])
s=max(df['polarity'])
m=min(df['polarity'])
avg=sum(df['polarity'])/(len(df['polarity']))

In [20]:
pos,neg=0,0
for i in df['polarity']:
    if i>0:
        pos+=1
    else:
        neg+=1
print(pos/l*100)
print(neg/l*100)

47.7846124273151
52.215387572684904


In [21]:
df.drop_duplicates(subset='text')
df.head()
df.shape

(41618, 3)

In [22]:
df['text'].to_csv('sentimental_analysis.csv',index=False)

In [23]:
!pip install nbconvert  # already-installed (most probably)
!apt install pandoc  # already-installed
!apt install texlive-xetex -y  # this'll take a long time

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (1.19.2.4~dfsg-1build4).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
texlive-xetex is already the newest version (2017.20180305-1).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.


In [26]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [28]:
nb_full_path = os.path.join(os.getcwd(), nb_name)

In [29]:
nb_name

'__notebook_source__.ipynb'

In [30]:
!jupyter nbconvert --execute --to pdf __notebook_source__.ipynb

[NbConvertApp] Converting notebook __notebook_source__.ipynb to pdf
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 20440 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 6595 bytes to __notebook_source__.pdf


In [32]:
!curl --upload-file __notebook_source__.pdf https://transfer.sh/notebook.pdf

                                  _   _ ____  _
  Project                     ___| | | |  _ \| |
                             / __| | | | |_) | |
                            | (__| |_| |  _ <| |___
                             \___|\___/|_| \_\_____|

NAME
       curl - transfer a URL

SYNOPSIS
       curl [options / URLs]

DESCRIPTION
       curl  is  a tool to transfer data from or to a server, using one of the
       supported protocols (DICT, FILE, FTP, FTPS, GOPHER, HTTP, HTTPS,  IMAP,
       IMAPS,  LDAP,  LDAPS,  POP3,  POP3S,  RTMP, RTSP, SCP, SFTP, SMB, SMBS,
       SMTP, SMTPS, TELNET and TFTP). The command is designed to work  without
       user interaction.

       curl offers a busload of useful tricks like proxy support, user authen-
       tication, FTP upload, HTTP post, SSL connections, cookies, file  trans-
       fer  resume,  Metalink,  and more. As you will see below, the number of
       features will make your head spin!

       curl is powered by  libcurl  for 